In [1]:
from google.cloud import bigquery
from google.cloud import aiplatform as vertexai
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, precision_score, recall_score,accuracy_score

In [2]:
data_df = pd.read_csv("201701.csv")

In [3]:
data_df.head()

,Unnamed: 0,fullVisitorId,visitId,date,visitNumber,channelGrouping,device_category,operating_system,traffic_source,traffic_medium,...,hostname,pageTitle,pagePathLevel1,pagePathLevel2,pagePathLevel3,pagePathLevel4,productSKU,v2ProductName,v2ProductCategory,addToCart
0,0,7178827723010268749,1485543372,20170127,2,Organic Search,mobile,iOS,(direct),(none),...,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,NaN,NaN,GGOEYOLR080599,YouTube Notebook and Pen Set,(not set),0
1,1,7178827723010268749,1485543372,20170127,2,Organic Search,mobile,iOS,(direct),(none),...,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,NaN,NaN,GGOEGAAX0317,YouTube Men's Short Sleeve Hero Tee White,(not set),0
2,2,7829316156760556769,1485577692,20170127,1,Direct,mobile,Android,(direct),(none),...,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,NaN,NaN,GGOEGAAX0613,Google Onesie Green,(not set),0
3,3,7829316156760556769,1485577692,20170127,1,Direct,mobile,Android,(direct),(none),...,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,NaN,NaN,GGOEGAAX0612,Google Onesie Royal Blue,(not set),0
4,4,7829316156760556769,1485577692,20170127,1,Direct,mobile,Android,(direct),(none),...,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,NaN,NaN,GGOEGAAX0611,Google Onesie Red,(not set),0


In [17]:
data_df.describe()

,Unnamed: 0,fullVisitorId,visitId,date,visitNumber,bounces,hits,newVisits,time_on_site,visits,pageviews_per_session,hitTime,hour,minute,addToCart
count,1.163110e+06,1.163110e+06,1.163110e+06,1.163110e+06,1.163110e+06,98442.0,1.163110e+06,838992.0,1.064132e+06,1163110.0,1.163101e+06,1.163110e+06,1.163110e+06,1.163110e+06,1.163110e+06
mean,5.815545e+05,4.507370e+18,1.484640e+09,2.017012e+07,2.650558e+00,1.0,2.113414e+01,1.0,6.469680e+02,1.0,1.659116e+01,2.793520e+05,1.220085e+01,2.980660e+01,6.312387e-03
std,3.357611e+05,3.075025e+18,7.744452e+05,8.957165e+00,1.269079e+01,0.0,3.421746e+01,0.0,9.064767e+02,0.0,2.404748e+01,5.265538e+05,6.025066e+00,1.725117e+01,7.919941e-02
min,0.000000e+00,4.541792e+13,1.483257e+09,2.017010e+07,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.907772e+05,1.584486e+18,1.483984e+09,2.017011e+07,1.000000e+00,1.0,5.000000e+00,1.0,1.210000e+02,1.0,4.000000e+00,1.976600e+04,8.000000e+00,1.500000e+01,0.000000e+00
50%,5.815545e+05,4.372836e+18,1.484683e+09,2.017012e+07,1.000000e+00,1.0,1.100000e+01,1.0,3.270000e+02,1.0,1.000000e+01,9.560400e+04,1.200000e+01,3.000000e+01,0.000000e+00
75%,8.723318e+05,7.243041e+18,1.485320e+09,2.017012e+07,2.000000e+00,1.0,2.500000e+01,1.0,8.120000e+02,1.0,2.000000e+01,3.036410e+05,1.700000e+01,4.500000e+01,0.000000e+00
max,1.163109e+06,9.999978e+18,1.485936e+09,2.017013e+07,3.170000e+02,1.0,5.000000e+02,1.0,9.772000e+03,1.0,3.400000e+02,9.772087e+06,2.300000e+01,5.900000e+01,1.000000e+00


In [18]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163110 entries, 0 to 1163109
Data columns (total 33 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1163110 non-null  int64  
 1   fullVisitorId          1163110 non-null  uint64 
 2   visitId                1163110 non-null  int64  
 3   date                   1163110 non-null  int64  
 4   visitNumber            1163110 non-null  int64  
 5   channelGrouping        1163110 non-null  object 
 6   device_category        1163110 non-null  object 
 7   operating_system       1163110 non-null  object 
 8   traffic_source         1163110 non-null  object 
 9   traffic_medium         1163110 non-null  object 
 10  traffic_campaign       1163110 non-null  object 
 11  country                1163110 non-null  object 
 12  city                   1163110 non-null  object 
 13  bounces                98442 non-null    float64
 14  hits              

In [19]:
data_df.isnull().sum()

Unnamed: 0                     0
fullVisitorId                  0
visitId                        0
date                           0
visitNumber                    0
channelGrouping                0
device_category                0
operating_system               0
traffic_source                 0
traffic_medium                 0
traffic_campaign               0
country                        0
city                           0
bounces                  1064668
hits                           0
newVisits                 324118
time_on_site               98978
visits                         0
pageviews_per_session          9
hitTime                        0
hour                           0
minute                         0
pagePath                       0
hostname                       0
pageTitle                    138
pagePathLevel1                 0
pagePathLevel2             71656
pagePathLevel3            297677
pagePathLevel4            815550
productSKU                     0
v2ProductN

In [20]:



data_df['bounces'] = data_df['bounces'].fillna(0)


In [21]:
threshold = 0.8 * len(data_df)  # Calculate 80% of the total number of rows

# Drop columns with more than 80% null values
data_df.dropna(thresh=threshold, axis=1, inplace=True)



In [22]:
data_df.drop('Unnamed: 0', axis=1, inplace=True)
data_df.drop('visitId', axis=1, inplace=True)
data_df.drop('date',axis=1, inplace=True)

In [23]:
unique_values_all = {}
for col in data_df.columns:
    unique_values_all[col] = data_df[col].unique()

# Print unique values for all columns
for col, unique_values in unique_values_all.items():
    print(f"Unique values in column '{col}': {unique_values}")

Unique values in column 'fullVisitorId': [7178827723010268749 7829316156760556769 5705504963778918068 ...
  986125786882084600 2786280313927380141 5490587536093604680]
Unique values in column 'visitNumber': [  2   1   7   5   4  39   3  13  11   8   9  10  12  18  21  23  31  27
  67   6  16  44  14  51  19  85  17  15  29  28  22  62  25  73 100  24
  47  41  20  36  88  33  59  30 178 317  48  50 160  52  32 312 313 314
 224 225  63  42 216 218  40 217  68 106  54 159 135 315 133  46  45 299
  49 300 303 302 108  75 309  99  84 310 311  26 215 294  58 214 293  35
 176  34  37  53 295 297 298  83  64 296 219  56  86  76 194 226 227 158
 103  43  69 230 229 221 228  66 110  87 102 231  71 101 291  55 105  38
 193  82 192 213  57 292 191  65  74 109 222]
Unique values in column 'channelGrouping': ['Organic Search' 'Direct' 'Paid Search' 'Referral' 'Social' 'Affiliates'
 'Display' '(Other)']
Unique values in column 'device_category': ['mobile' 'desktop' 'tablet']
Unique values in column 

In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163110 entries, 0 to 1163109
Data columns (total 33 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1163110 non-null  int64  
 1   fullVisitorId          1163110 non-null  uint64 
 2   visitId                1163110 non-null  int64  
 3   date                   1163110 non-null  int64  
 4   visitNumber            1163110 non-null  int64  
 5   channelGrouping        1163110 non-null  object 
 6   device_category        1163110 non-null  object 
 7   operating_system       1163110 non-null  object 
 8   traffic_source         1163110 non-null  object 
 9   traffic_medium         1163110 non-null  object 
 10  traffic_campaign       1163110 non-null  object 
 11  country                1163110 non-null  object 
 12  city                   1163110 non-null  object 
 13  bounces                98442 non-null    float64
 14  hits              

In [27]:
categorical_features=data_df.select_dtypes(include=['object'])

In [28]:
categorical_features

,channelGrouping,device_category,operating_system,traffic_source,traffic_medium,traffic_campaign,country,city,pagePath,hostname,pageTitle,pagePathLevel1,pagePathLevel2,productSKU,v2ProductName,v2ProductCategory
0,Organic Search,mobile,iOS,(direct),(none),(not set),Austria,not available in demo dataset,/asearch.html,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,GGOEYOLR080599,YouTube Notebook and Pen Set,(not set)
1,Organic Search,mobile,iOS,(direct),(none),(not set),Austria,not available in demo dataset,/asearch.html,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,GGOEGAAX0317,YouTube Men's Short Sleeve Hero Tee White,(not set)
2,Direct,mobile,Android,(direct),(none),(not set),United States,not available in demo dataset,/asearch.html,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,GGOEGAAX0613,Google Onesie Green,(not set)
3,Direct,mobile,Android,(direct),(none),(not set),United States,not available in demo dataset,/asearch.html,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,GGOEGAAX0612,Google Onesie Royal Blue,(not set)
4,Direct,mobile,Android,(direct),(none),(not set),United States,not available in demo dataset,/asearch.html,shop.googlemerchandisestore.com,Store search results,/asearch.html,NaN,GGOEGAAX0611,Google Onesie Red,(not set)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163105,Social,desktop,Macintosh,facebook.com,referral,(not set),Netherlands,not available in demo dataset,/google+redesign/office,shop.googlemerchandisestore.com,Office | Google Merchandise Store,/google+redesign/,/office,GGOEGOAR013099,Google Stylus Pen w/ LED Light,Home/Office/
1163106,Social,desktop,Macintosh,facebook.com,referral,(not set),Netherlands,not available in demo dataset,/google+redesign/office,shop.googlemerchandisestore.com,Office | Google Merchandise Store,/google+redesign/,/office,GGOEGOAQ018099,Pen Pencil & Highlighter Set,Home/Office/
1163107,Social,desktop,Macintosh,facebook.com,referral,(not set),Netherlands,not available in demo dataset,/google+redesign/office,shop.googlemerchandisestore.com,Office | Google Merchandise Store,/google+redesign/,/office,GGOEGOAA017199,Rubber Grip Ballpoint Pen 4 Pack,Home/Office/
1163108,Social,desktop,Macintosh,facebook.com,referral,(not set),Netherlands,not available in demo dataset,/google+redesign/shop+by+brand/google,shop.googlemerchandisestore.com,Google | Shop by Brand | Google Merchandise Store,/google+redesign/,/shop+by+brand/,GGOEGAAX0104,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Shop by Brand/Google/


In [30]:
num_featur=data_df.select_dtypes(include=['int','float'])

In [15]:
num_featur

,visitId,visitNumber,hits,time_on_site,visits,pageviews_per_session,hitTime,hour,minute,addToCart
0,1485543372,2,1,NaN,1,1.0,0,10,56,0
1,1485543372,2,1,NaN,1,1.0,0,10,56,0
2,1485577692,1,1,NaN,1,1.0,0,20,28,0
3,1485577692,1,1,NaN,1,1.0,0,20,28,0
4,1485577692,1,1,NaN,1,1.0,0,20,28,0
...,...,...,...,...,...,...,...,...,...,...
1163105,1484641572,1,16,546.0,1,16.0,396422,0,32,0
1163106,1484641572,1,16,546.0,1,16.0,396422,0,32,0
1163107,1484641572,1,16,546.0,1,16.0,396422,0,32,0
1163108,1484641572,1,16,546.0,1,16.0,414214,0,33,0


In [32]:
encoded_df['addToCart']

0          0
1          0
2          0
3          0
4          0
          ..
1163105    0
1163106    0
1163107    0
1163108    0
1163109    0
Name: addToCart, Length: 1163110, dtype: int64